In [1]:
#%matplotlib notebook
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
import networkx as nx
from tqdm import tqdm
from sklearn.model_selection import train_test_split

%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


/data/user/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_bone_eeg_code_A_E_v/pyiomica_master/pyiomica/visibilityGraphAuxiliaryFunctions.py:91: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(cache=True)


In [2]:
data_path="/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/"

In [3]:
df_path=glob.glob(data_path+"***")

In [4]:
df_path

['/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/Set_A',
 '/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/Set_B',
 '/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/Set_C',
 '/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/Set_D',
 '/home/sahaj432/Desktop/dataset/Bonn_seizure_dataset/Set_E']

In [5]:
LABEL_C1 = 0
LABEL_C2 = 1

In [6]:
def load_data():
    data = []
    nbFiles = 0
    df=glob.glob(df_path[0]+"/***")
    for fname in tqdm(df):
        img = np.loadtxt(fname)
        data.append([img, np.array(LABEL_C1)])
        nbFiles+=1
        
    df1=glob.glob(df_path[-1]+"/***")
    for fname in tqdm(df1):
        img = np.loadtxt(fname)
        data.append([np.array(img), np.array(LABEL_C2)])
        nbFiles+=1
    
    return data

In [7]:
data = load_data()
print(len(data), "Files")

100%|██████████| 100/100 [00:01<00:00, 69.37it/s]

200 Files


In [8]:
from sklearn.utils import shuffle
data = shuffle(data,random_state=40)


X = np.array([d[0] for d in data])
Y = np.array([d[1] for d in data])



In [9]:
X.shape

(200, 4097)

In [10]:
def min_max_normalize(matrix):
    """
    Perform min-max normalization on a 2D array (matrix).
    """
    min_vals = np.min(matrix, axis=0)
    max_vals = np.max(matrix, axis=0)
    normalized = (matrix - min_vals) / (max_vals - min_vals)
    return normalized

In [11]:
X_n=np.array([min_max_normalize(i) for i in X])


In [12]:
X_temp=[]
Y_temp=[]
for i in range(len(X)):
        raw=X_n[i]
        X_temp.append(raw[:1024])
        X_temp.append(raw[1024:2048])
        X_temp.append(raw[2048:3072])
        X_temp.append(raw[3072:4096])
        
        Y_temp.extend(np.repeat(Y[i],4))

        

In [13]:
X_temp=np.array(X_temp)
Y_temp=np.array(Y_temp)

In [14]:
print(X_temp.shape)
print(Y_temp.shape)

(800, 1024)
(800,)


In [15]:
from sklearn.utils import shuffle
X_s,Y_s = shuffle(X_temp,Y_temp,random_state=40)


In [16]:
unique_elements, element_counts = np.unique(Y_s, return_counts=True)

print("Number of unique elements:", len(unique_elements))
print("Unique elements:", unique_elements)
print("Element counts:", element_counts)


Number of unique elements: 2
Unique elements: [0 1]
Element counts: [400 400]


In [17]:
def calculate_visibility(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "natural", weight='tan')
        vs_out=np.array(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [18]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [19]:
df=np.array_split(X_s, num_cores)


In [20]:
print(len(df[0]))

17


In [21]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_visibility, df)

pool.close()
pool.join()

In [22]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [23]:
print(len(x_adj))

800


In [24]:
x_adj=np.array(x_adj)

In [25]:
def graph_index_complexity(matrix):
    N = len(matrix) # Number of nodes
    E = np.sum(matrix > 0) / 2 # Number of edges (since matrix is symmetric for undirected graphs)
    
    if N == 0 or E == 0:
        return 0
    
    GIC = (E / N) * np.log(N)
    return GIC

In [26]:
def degree(data): #The node strengths (or weighted degrees) 
    return np.sum(data, axis=0).flatten()

In [27]:
def runall(data):
    temp1=[]
    for i in data:
        G = nx.from_numpy_array(i, parallel_edges=False)
        
        cpl=[nx.average_shortest_path_length(G)]
        ge=[nx.global_efficiency(G)]
        le=[nx.local_efficiency(G)]
        ac=[nx.degree_assortativity_coefficient(G)]
        d=degree(i).tolist()
        bc=list(nx.betweenness_centrality(G).values())
        cc=list(nx.clustering(G).values())
        temp1.append(cpl+ge+le+ac+d+bc+cc) #+cpl+ge+le+ac+cc
    return temp1

In [28]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
17


In [29]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(runall, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 246.24606704711914 seconds ---


In [30]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [31]:
x_g=np.array(x_g)


In [32]:
print(x_g.shape)

(800, 3076)


In [33]:
x_train, x_test, y_train, y_test = train_test_split(x_g, Y_s, test_size=0.30, random_state=42)


In [34]:
print(x_train.shape)
print(x_test.shape)

(560, 3076)
(240, 3076)


In [35]:
import tensorflow as tf


2023-08-12 07:47:53.056788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-12 07:47:53.056823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [36]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-12 07:48:18.169836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-12 07:48:18.169870: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-12 07:48:18.169893: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0175): /proc/driver/nvidia/version does not exist
2023-08-12 07:48:18.170232: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3076, 32)          128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1538, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1538, 32)          3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 769, 32)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 769, 32)           3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 384, 32)          0

In [38]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [39]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F2.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [40]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
18/18 [==============================] - 2s 52ms/step - loss: 0.8391 - acc: 0.5518 - val_loss: 0.4696 - val_acc: 0.8208
Epoch 2/30
18/18 [==============================] - 0s 23ms/step - loss: 0.4921 - acc: 0.7661 - val_loss: 0.3296 - val_acc: 0.8583
Epoch 3/30
18/18 [==============================] - 0s 22ms/step - loss: 0.2665 - acc: 0.9107 - val_loss: 0.2002 - val_acc: 0.9167
Epoch 4/30
18/18 [==============================] - 0s 23ms/step - loss: 0.1940 - acc: 0.9071 - val_loss: 0.1681 - val_acc: 0.9333
Epoch 5/30
18/18 [==============================] - 0s 25ms/step - loss: 0.1363 - acc: 0.9482 - val_loss: 0.1500 - val_acc: 0.9375
Epoch 6/30
18/18 [==============================] - 0s 24ms/step - loss: 0.0829 - acc: 0.9732 - val_loss: 0.1400 - val_acc: 0.9417
Epoch 7/30
18/18 [==============================] - 0s 23ms/step - loss: 0.0599 - acc: 0.9768 - val_loss: 0.1493 - val_acc: 0.9458
Epoch 8/30
18/18 [==============================] - 0s 23ms/step - loss: 0.0547 - a

18/18 [==============================] - 0s 25ms/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1352 - val_acc: 0.9625
Epoch 30/30
18/18 [==============================] - 0s 24ms/step - loss: 5.6918e-04 - acc: 1.0000 - val_loss: 0.1002 - val_acc: 0.9625


In [41]:
model3.save_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_bone_eeg_code_A_E_v/F2.h5')


In [42]:
model3.load_weights('/home/sahaj432/Desktop/code_organized/G_signal_feature_reduction_Project/New_bone_eeg_code_A_E_v/F2.h5')


In [43]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [44]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[109   2]
 [  7 122]]


In [45]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [46]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

96.25
96.23896501889224
96.25
96.34211624026698
